In [762]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [763]:
data = pd.read_excel('card transactions.xlsx')

In [764]:
# only use purchase type transactions
df = data.loc[data['Transtype'] == 'P']

In [765]:
df.shape

(96398, 10)

In [766]:
df.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud'],
      dtype='object')

In [767]:
df[df['Merch description'] == '0000000000000000000000']

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
83923,83924,5142182645,2010-10-28,8168600400097,0000000000000000000000,WA,98052.0,P,463.86,0
92542,92543,5142185554,2010-12-10,8168600400097,0000000000000000000000,WA,98052.0,P,127.87,0
92570,92571,5142185554,2010-12-10,8168600400097,0000000000000000000000,WA,98052.0,P,769.70,0


## Fix Missing Values

### Fix Missing Zipcode & Adjust some entries

In [768]:
import googlemaps
gmaps = googlemaps.Client(key='apikey')

**Step 1: fix missing zipcode with Merch state and Merch description**

In [769]:
# Step 1: fix missing zipcode with Merch state and Merch description
temp = df[(df['Merch zip'].isnull()) & (df['Merch state'].notnull())]
df['Merch zip'] = df['Merch zip'].fillna(0)

for i in temp.index:
    s = df.loc[i]['Merch description']
    result = " ".join(re.split("[^a-zA-Z]+", s))
    address = result + ' , ' + df.loc[i]['Merch state']
    location = gmaps.geocode(address)
    
    if len(location) == 0:   #some unrecognized address
        pass
    else:
        if location[0]['address_components'][-1]['types'] == ['postal_code']:
            zipcode = location[0]['address_components'][-1]['long_name']
            df.loc[df.index == i, 'Merch zip'] = zipcode
        else:
            zipcode = location[0]['address_components'][-2]['long_name']
            df.loc[df.index == i, 'Merch zip'] = zipcode          

**Step 2: fix zipcode that is not 5 digits**

In [770]:
# Step 2: fix zipcode that is not 5 digits
df['Merch zip'] = df['Merch zip'].astype(str)

In [771]:
not_5_zip = []

for i,j in zip(df['Merch zip'], df.index):
    if i[:5].isdecimal() == False:
        not_5_zip.append(j)
    else:
        continue

In [772]:
# 4 digit zipcode may start with 0
temp = df[df.index.isin(not_5_zip)]['Merch zip']

four_zip = []

for i,j in zip(temp.str[:4],temp.index):
    if i.isdecimal():
        four_zip.append(j)
    else:
        continue

In [773]:
# states that zipcode start with 0 
state_zip_0 = ['CT', 'MA', 'ME', 'NH', 'NJ', 'PR', 'RI', 'VT', 'VI']

for i in four_zip:
    if df.loc[i]['Merch state'] in state_zip_0:
        new = '0' + df.loc[i]['Merch zip']
        df.loc[df.index == i, 'Merch zip'] = new

**Step 3: others**

In [774]:
others = list(set(not_5_zip) - set(four_zip))

In [775]:
temp = df[(df.index.isin(others)) & (df['Merch state'].notnull())]
not_found = []

for i in temp.index:
    s = df.loc[i]['Merch description']
    result = " ".join(re.split("[^a-zA-Z]+", s))
    address = result + ' , ' + df.loc[i]['Merch state']
    location = gmaps.geocode(address)
    
    if len(location) == 0:   #some unrecognized address
        not_found.append(i)
        pass
    else:
        if location[0]['address_components'][-1]['types'] == ['postal_code']:
            zipcode = location[0]['address_components'][-1]['long_name']
            df.loc[df.index == i, 'Merch zip'] = zipcode
        else:
            zipcode = location[0]['address_components'][-2]['long_name']
            df.loc[df.index == i, 'Merch zip'] = zipcode

In [776]:
df[df.index.isin(not_found)]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
10486,10487,5142227402,2010-02-14,NaN,NORTHWEST ECONOMETRICS,BC,0.0,P,577.00,0
13934,13935,5142120389,2010-02-28,NaN,LANGEVIN LEARN SERV(US)IN,ON,0.0,P,259.35,0
20802,20803,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,0.0,P,399.00,0
20866,20867,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,0.0,P,199.00,0
21063,21064,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,0.0,P,359.00,0
21102,21103,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,0.0,P,199.00,0
22444,22445,5142120389,2010-03-28,NaN,LANGEVIN LEARN SERV(US)IN,ON,0.0,P,399.00,0
30626,30627,5142182005,2010-04-27,6497600064127,JONES CHROMATOGRAPHY,CO,0.0,P,1479.00,0
34906,34907,5142189108,2010-05-11,NaN,GOC-CCOHS,ON,0.0,P,365.00,0
39956,39957,5142189135,2010-05-29,9006005990206,LYNNON BIOSOFT,PQ,0.0,P,468.00,0


In [777]:
# Lower postal code from online source: 
# https://www.swiftestimator.com/SE7Help/CE/help/Miscellaneous/Using_Canadian_Postal_Codes.htm
zip_online = {'ON':'K0G 0A0', 'CO':'80218', 'TX': '75001', 'NY': '10081', 'NS': 'B0A 0A0', 
              'PQ': 'G0A 0A0', 'BC': 'V5K 0A1', 'WA': '98003'}

In [778]:
zip_online.keys()

dict_keys(['ON', 'CO', 'TX', 'NY', 'NS', 'PQ', 'BC', 'WA'])

In [779]:
for i in not_found:
    for j in zip_online.keys():
        if df.loc[i]['Merch state'] == j:
            df.loc[df.index == i,'Merch zip'] = zip_online[j]

In [780]:
df[df.index.isin(not_found)]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
10486,10487,5142227402,2010-02-14,NaN,NORTHWEST ECONOMETRICS,BC,V5K 0A1,P,577.00,0
13934,13935,5142120389,2010-02-28,NaN,LANGEVIN LEARN SERV(US)IN,ON,K0G 0A0,P,259.35,0
20802,20803,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,K0G 0A0,P,399.00,0
20866,20867,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,K0G 0A0,P,199.00,0
21063,21064,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,K0G 0A0,P,359.00,0
21102,21103,5142196337,2010-03-22,NaN,LANGEVIN LEARNING SERVICE,ON,K0G 0A0,P,199.00,0
22444,22445,5142120389,2010-03-28,NaN,LANGEVIN LEARN SERV(US)IN,ON,K0G 0A0,P,399.00,0
30626,30627,5142182005,2010-04-27,6497600064127,JONES CHROMATOGRAPHY,CO,80218,P,1479.00,0
34906,34907,5142189108,2010-05-11,NaN,GOC-CCOHS,ON,K0G 0A0,P,365.00,0
39956,39957,5142189135,2010-05-29,9006005990206,LYNNON BIOSOFT,PQ,G0A 0A0,P,468.00,0


### Fix Missing state and corresponding zip

In [781]:
df[df['Merch state'].isnull()]['Merch description'].value_counts()

RETAIL CREDIT ADJUSTMENT     357
RETAIL DEBIT ADJUSTMENT      295
ELSEVIER SCIENCE BV           58
XFR F/VILLARREAL 12637808     19
CREDIT PURCHASE BALANCE       19
                            ... 
WWW.ALTA4.DE                   1
IDEAL SOFTWARE                 1
TOMORROW MAGAZINE              1
VIEWEG FRIEDRICH & SOHN        1
CAMPDEN PUBLISHING LTD         1
Name: Merch description, Length: 151, dtype: int64

In [782]:
df[df['Merch state'].isnull()]['Fraud'].value_counts()

0    1018
1       3
Name: Fraud, dtype: int64

**Step 1: Fix missing state and zip for two adjustments with previous records if cardnum and description are same**

In [783]:
temp1 = df[df['Merch description'].isin(['RETAIL CREDIT ADJUSTMENT', 'RETAIL DEBIT ADJUSTMENT']) & 
          df['Merch state'].isnull()]

temp2 = df[df['Merch description'].isin(['RETAIL CREDIT ADJUSTMENT', 'RETAIL DEBIT ADJUSTMENT']) & 
          df['Merch state'].notnull()]


for i in temp1.index:
    for j in temp2.index:
        if (df.loc[i]['Cardnum'] == df.loc[j]['Cardnum']) & (
            df.loc[i]['Merch description'] == df.loc[j]['Merch description']):
            df.loc[df.index==i,'Merch state'] = df.loc[j]['Merch state']
            df.loc[df.index==i,'Merch zip'] = df.loc[j]['Merch zip']

**Step 2: Fix missing state and zip for two adjustments with most state and zip appeared for these two adjustments**

In [784]:
df[df['Merch description'].isin(['RETAIL CREDIT ADJUSTMENT', 'RETAIL DEBIT ADJUSTMENT']) & 
          df['Merch state'].notnull()][['Merch state','Merch zip']].value_counts()

Merch state  Merch zip           
VA           23229                   48
GA           30326                    9
DC           District of Columbia     9
VA           23605                    8
CA           03103                    6
DC           20009                    5
GA           43512                    4
MD           21076                    3
TX           76133                    1
dtype: int64

In [785]:
temp = df[df['Merch description'].isin(['RETAIL CREDIT ADJUSTMENT', 'RETAIL DEBIT ADJUSTMENT']) & 
          df['Merch state'].isnull()]

for i in temp.index:
    df.loc[df.index == i, 'Merch state'] = 'VA'
    df.loc[df.index == i, 'Merch zip'] = '23229'

**Step 3: other missing, fix with mode**

In [798]:
df[df['Merch state'].isnull()]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
931,932,5142214551,2010-01-06,6176269,MUNKSGAARDS FORLAG,NaN,0.0,P,1790.00,0
1275,1276,5142158180,2010-01-07,NaN,BLACKWELL SCIENCE,NaN,0.0,P,584.90,0
1402,1403,5142159706,2010-01-07,0,INTERACTIVE SOFTWARE S,NaN,0.0,P,460.00,0
1561,1562,5142214551,2010-01-10,NaN,ELSEVIER SCIENCE BV,NaN,0.0,P,1053.00,0
2042,2043,5142214551,2010-01-11,6176269,MUNKSGAARDS FORLAG,NaN,0.0,P,3100.00,0
...,...,...,...,...,...,...,...,...,...,...
91980,91981,5142111372,2010-12-08,0,CATCHWORD LTD,NaN,0.0,P,19.39,0
92921,92922,5142197979,2010-12-13,NaN,IARC PRESS VAD 2184997,NaN,0.0,P,260.00,0
93146,93147,5142224426,2010-12-14,8457033078615,GREEN LEAF PUBLISHING,NaN,0.0,P,24.95,0
93760,93761,5142195916,2010-12-15,0,REED BUSINESS PUBLISHI,NaN,0.0,P,140.00,0


In [788]:
df[df['Merch state'].isnull()]['Fraud'].value_counts()

0    367
1      2
Name: Fraud, dtype: int64

In [799]:
df.loc[df['Fraud'] == 0][['Merch state','Merch zip']].mode()

,Merch state,Merch zip
0,TN,38118.0


In [800]:
df.loc[df['Fraud'] == 1][['Merch state','Merch zip']].mode()

,Merch state,Merch zip
0,WA,98101.0


In [801]:
# fix other missing state and missing zip by mode in each class
temp = df[df['Merch state'].isnull()]

for i in temp.index:
    if df.loc[i]['Fraud'] == 0:
        df.loc[df.index == i, 'Merch state'] = 'TN'
        df.loc[df.index == i, 'Merch zip'] = '38118'
    else:
        df.loc[df.index == i, 'Merch state'] = 'WA'
        df.loc[df.index == i, 'Merch zip'] = '98101'      

In [802]:
df[df['Merch state'].isnull()].sum()

Recnum               0.0
Cardnum              0.0
Date                 0.0
Merchnum             0.0
Merch description    0.0
Merch state          0.0
Merch zip            0.0
Transtype            0.0
Amount               0.0
Fraud                0.0
dtype: float64

In [803]:
df[df['Merch zip'].isnull()].sum()

Recnum               0.0
Cardnum              0.0
Date                 0.0
Merchnum             0.0
Merch description    0.0
Merch state          0.0
Merch zip            0.0
Transtype            0.0
Amount               0.0
Fraud                0.0
dtype: float64

In [804]:
len(df[df['Merch zip'] == 0])

0

### Fix missing Merchnum

In [805]:
df.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3199
Merch description       0
Merch state             0
Merch zip               0
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [806]:
# make the missing to '0' and then fix together
df['Merchnum'] = df['Merchnum'].fillna('0')

In [819]:
# fill missing and bad Merchnum with values from other records that 
# share the same Merch description, Merch state and Merch zip
temp1 = df[df['Merchnum'].isin(['0'])] 
temp2 = df[~df['Merchnum'].isin(['0'])] 
temp3 = df.groupby(['Merch description', 'Merch state', 'Merch zip'])

for i in temp1.index: 
    for j in temp3.groups[df.loc[i]['Merch description'], df.loc[i]['Merch state'], df.loc[i]['Merch zip']]:
        if j in temp2.index:
            df.loc[df.index == i, 'Merchnum'] = df.loc[j]['Merchnum']

In [820]:
df[df['Merchnum'].isin(['0'])]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
201,202,5142140436,2010-01-03,0,MONTGOMERY COLLEGE-PHONE,MD,20850.0,P,266.00,0
262,263,5142257575,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,320.00,0
272,273,5142124791,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,970.00,0
400,401,5142276099,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,82.59,0
476,477,5142267793,2010-01-05,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,17.59,0
...,...,...,...,...,...,...,...,...,...,...
96621,96622,5142253938,2010-12-30,0,OREGON ENVIRONMENTAL COUN,OR,97204.0,P,40.00,0
96651,96652,5142151962,2010-12-30,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,125.00,0
96689,96690,5142166019,2010-12-30,0,ADVANCED SEC & CONTROL,GA,30340.0,P,70.50,0
96720,96721,5142178272,2010-12-31,0,TRULOGIC SYSTEMS,ON,01930,P,343.42,0


In [826]:
# fill missing and bad Merchnum with values from other records that share the same Merch description, Merch state
temp1 = df[df['Merchnum'].isin(['0'])] 
temp2 = df[~df['Merchnum'].isin(['0'])] 
temp3 = df.groupby(['Merch description', 'Merch state'])

for i in temp1.index: 
    for j in temp3.groups[df.loc[i]['Merch description'], df.loc[i]['Merch state']]:
        if j in temp2.index:
            df.loc[df.index == i, 'Merchnum'] = df.loc[j]['Merchnum']

In [827]:
df[df['Merchnum'].isin(['0'])]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
201,202,5142140436,2010-01-03,0,MONTGOMERY COLLEGE-PHONE,MD,20850,P,266.00,0
262,263,5142257575,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,320.00,0
272,273,5142124791,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,970.00,0
400,401,5142276099,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,82.59,0
476,477,5142267793,2010-01-05,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,17.59,0
...,...,...,...,...,...,...,...,...,...,...
96621,96622,5142253938,2010-12-30,0,OREGON ENVIRONMENTAL COUN,OR,97204,P,40.00,0
96651,96652,5142151962,2010-12-30,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,125.00,0
96689,96690,5142166019,2010-12-30,0,ADVANCED SEC & CONTROL,GA,30340,P,70.50,0
96720,96721,5142178272,2010-12-31,0,TRULOGIC SYSTEMS,ON,01930,P,343.42,0


In [830]:
# fill missing and bad Merchnum with values from other records that share the same Merch description
temp1 = df[df['Merchnum'].isin(['0'])] 
temp2 = df[~df['Merchnum'].isin(['0'])] 
temp3 = df.groupby(['Merch description'])

for i in temp1.index: 
    for j in temp3.groups[df.loc[i]['Merch description']]:
        if j in temp2.index:
            df.loc[df.index == i, 'Merchnum'] = df.loc[j]['Merchnum']

In [831]:
df[df['Merchnum'].isin(['0'])]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
201,202,5142140436,2010-01-03,0,MONTGOMERY COLLEGE-PHONE,MD,20850,P,266.00,0
262,263,5142257575,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,320.00,0
272,273,5142124791,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,970.00,0
400,401,5142276099,2010-01-04,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,82.59,0
476,477,5142267793,2010-01-05,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,17.59,0
...,...,...,...,...,...,...,...,...,...,...
96621,96622,5142253938,2010-12-30,0,OREGON ENVIRONMENTAL COUN,OR,97204,P,40.00,0
96651,96652,5142151962,2010-12-30,0,RETAIL DEBIT ADJUSTMENT,VA,23229,P,125.00,0
96689,96690,5142166019,2010-12-30,0,ADVANCED SEC & CONTROL,GA,30340,P,70.50,0
96720,96721,5142178272,2010-12-31,0,TRULOGIC SYSTEMS,ON,01930,P,343.42,0


In [832]:
# fill others with "Unknown"
df['Merchnum'] = df['Merchnum'].replace('0','unknown')

### Final Dataset

In [823]:
# remove the single large amount transaction
df = df.drop(index = [52714])

In [824]:
# change zipcode to 5 digits if it is U.S. code
for i in df.index:
    if df.loc[i]['Merch zip'][:5].isdecimal():
        df.loc[df.index == i, 'Merch zip'] = df.loc[i]['Merch zip'][:5]

In [833]:
df.isnull().sum()

Recnum               0
Cardnum              0
Date                 0
Merchnum             0
Merch description    0
Merch state          0
Merch zip            0
Transtype            0
Amount               0
Fraud                0
dtype: int64

In [834]:
#df.to_csv('clean card transaction.csv')